In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import time  # To track the elapsed time
import matplotlib.colors as mcolors
import concurrent.futures
import re
import string
from scipy.stats import linregress
import matplotlib.patches as mpatches
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [2]:
# Function to standardize county names: lowercase and remove 'county', 'parish', and text in parentheses
# Updated function to clean and standardize county names
def clean_county_name(name):
    # Convert to lowercase
    name = name.lower()
    # Remove 'county', 'parish', and any content within parentheses
    name = re.sub(r'\b(county|parish)\b', '', name)
    name = re.sub(r'\(.*?\)', '', name)
    # Remove punctuation
    name = name.translate(str.maketrans('', '', string.punctuation))
    # Remove extra whitespace
    name = name.strip()
    name = re.sub(r'\s+', ' ', name)  # Replace multiple spaces with a single space
    return name

# FIPS Code Load

In [3]:
# Read the file line by line
with open('../Local_Data/fips_codes.txt', 'r') as file:
    lines = file.readlines()

# Step 1: Identify where state-level data starts and ends
state_section_start = 0
county_section_start = 0

for i, line in enumerate(lines):
    if "state-level" in line.lower():
        state_section_start = i + 2  # State data starts after the "state-level" heading
    if "county-level" in line.lower():
        county_section_start = i + 2  # County data starts after the "county-level" heading
        break

# Manually create the dictionary mapping state FIPS codes to state names
state_fips_dict = {
    "01": "ALABAMA",
    "02": "ALASKA",
    "04": "ARIZONA",
    "05": "ARKANSAS",
    "06": "CALIFORNIA",
    "08": "COLORADO",
    "09": "CONNECTICUT",
    "10": "DELAWARE",
    "11": "DISTRICT OF COLUMBIA",
    "12": "FLORIDA",
    "13": "GEORGIA",
    "15": "HAWAII",
    "16": "IDAHO",
    "17": "ILLINOIS",
    "18": "INDIANA",
    "19": "IOWA",
    "20": "KANSAS",
    "21": "KENTUCKY",
    "22": "LOUISIANA",
    "23": "MAINE",
    "24": "MARYLAND",
    "25": "MASSACHUSETTS",
    "26": "MICHIGAN",
    "27": "MINNESOTA",
    "28": "MISSISSIPPI",
    "29": "MISSOURI",
    "30": "MONTANA",
    "31": "NEBRASKA",
    "32": "NEVADA",
    "33": "NEW HAMPSHIRE",
    "34": "NEW JERSEY",
    "35": "NEW MEXICO",
    "36": "NEW YORK",
    "37": "NORTH CAROLINA",
    "38": "NORTH DAKOTA",
    "39": "OHIO",
    "40": "OKLAHOMA",
    "41": "OREGON",
    "42": "PENNSYLVANIA",
    "44": "RHODE ISLAND",
    "45": "SOUTH CAROLINA",
    "46": "SOUTH DAKOTA",
    "47": "TENNESSEE",
    "48": "TEXAS",
    "49": "UTAH",
    "50": "VERMONT",
    "51": "VIRGINIA",
    "53": "WASHINGTON",
    "54": "WEST VIRGINIA",
    "55": "WISCONSIN",
    "56": "WYOMING"
}

# Convert the dictionary to a DataFrame for easy merging
state_df = pd.DataFrame(list(state_fips_dict.items()), columns=['State FIPS', 'State Name'])

# Step 1: Split the data into sections
state_section_start = 0
county_section_start = 0

for i, line in enumerate(lines):
    if "state-level" in line.lower():
        state_section_start = i + 2  # Skip the header lines
    if "county-level" in line.lower():
        county_section_start = i + 2  # Skip the header lines
        break

state_lines = lines[state_section_start:county_section_start-2]  # State data lines
county_lines = lines[county_section_start:]  # County data line

# Step 3: Parse county data
county_fips = []
county_name = []

for line in county_lines:
    line = line.strip()
    if len(line) > 12 and line[:5].isdigit():
        county_fips_code = line[:5].strip()
        county_fips_name = line[12:].strip()
        county_fips.append(county_fips_code)
        county_name.append(county_fips_name)

county_df = pd.DataFrame({
    'countyCode': county_fips,
    'County Name': county_name
})

county_df['State FIPS'] = county_df['countyCode'].str[:2]  # Extract state FIPS from the countyCode

# Assuming county_df is already created and correctly populated:
fips_df = county_df.merge(state_df, on='State FIPS')
fips_df['County Name'] = fips_df['County Name'].str.upper()

# Final output
print("\nMerged FIPS DataFrame:")
print(fips_df.head())


Merged FIPS DataFrame:
  countyCode     County Name State FIPS State Name
0      01000         ALABAMA         01    ALABAMA
1      01001  AUTAUGA COUNTY         01    ALABAMA
2      01003  BALDWIN COUNTY         01    ALABAMA
3      01005  BARBOUR COUNTY         01    ALABAMA
4      01007     BIBB COUNTY         01    ALABAMA


# Load Aid Data

In [4]:
HA_owners = pd.read_csv('../Local_Data/FEMA_Disaster_Asst_Data/HousingAssistanceOwners.csv')
HA_renters = pd.read_csv('../Local_Data/FEMA_Disaster_Asst_Data/HousingAssistanceRenters.csv')

/tmp/ipykernel_2875/2368396277.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  HA_owners = pd.read_csv('../Game/FEMA_Disaster_Asst_Data/HousingAssistanceOwners.csv')
/tmp/ipykernel_2875/2368396277.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  HA_renters = pd.read_csv('../Game/FEMA_Disaster_Asst_Data/HousingAssistanceRenters.csv')


In [5]:
# Full mapping of state abbreviations to names
state_abbr_to_name = {
    "AL": "ALABAMA", "AK": "ALASKA", "AZ": "ARIZONA", "AR": "ARKANSAS", "CA": "CALIFORNIA",
    "CO": "COLORADO", "CT": "CONNECTICUT", "DE": "DELAWARE", "FL": "FLORIDA", "GA": "GEORGIA",
    "HI": "HAWAII", "ID": "IDAHO", "IL": "ILLINOIS", "IN": "INDIANA", "IA": "IOWA",
    "KS": "KANSAS", "KY": "KENTUCKY", "LA": "LOUISIANA", "ME": "MAINE", "MD": "MARYLAND",
    "MA": "MASSACHUSETTS", "MI": "MICHIGAN", "MN": "MINNESOTA", "MS": "MISSISSIPPI",
    "MO": "MISSOURI", "MT": "MONTANA", "NE": "NEBRASKA", "NV": "NEVADA", "NH": "NEW HAMPSHIRE",
    "NJ": "NEW JERSEY", "NM": "NEW MEXICO", "NY": "NEW YORK", "NC": "NORTH CAROLINA",
    "ND": "NORTH DAKOTA", "OH": "OHIO", "OK": "OKLAHOMA", "OR": "OREGON", "PA": "PENNSYLVANIA",
    "RI": "RHODE ISLAND", "SC": "SOUTH CAROLINA", "SD": "SOUTH DAKOTA", "TN": "TENNESSEE",
    "TX": "TEXAS", "UT": "UTAH", "VT": "VERMONT", "VA": "VIRGINIA", "WA": "WASHINGTON",
    "WV": "WEST VIRGINIA", "WI": "WISCONSIN", "WY": "WYOMING"
}

# Map state abbreviations to full names in HA_owners
HA_owners['State Name'] = HA_owners['state'].map(state_abbr_to_name)
HA_renters['State Name'] = HA_renters['state'].map(state_abbr_to_name)

# Apply cleaning to county columns in both dataframes
fips_df['County Name'] = fips_df['County Name'].apply(clean_county_name)
HA_owners['county'] = HA_owners['county'].apply(clean_county_name)
HA_renters['county'] = HA_renters['county'].apply(clean_county_name)


# Merge on cleaned 'County Name' and 'State Name' to find matches
HA_owners_merged = HA_owners.merge(fips_df[['County Name', 'State Name']], 
                            left_on=['county', 'State Name'], 
                            right_on=['County Name', 'State Name'], 
                            how='outer', 
                            indicator=True)
HA_renters_merged = HA_renters.merge(fips_df[['County Name', 'State Name']], 
                            left_on=['county', 'State Name'], 
                            right_on=['County Name', 'State Name'], 
                            how='outer', 
                            indicator=True)

# Define a set of valid US state abbreviations (50 states only)
us_states = {
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA", "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME",
    "MD", "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA",
    "RI", "SC", "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
}

# Get matching and non-matching pairs
matches = HA_owners_merged[HA_owners_merged['_merge'] == 'both']
non_matches = HA_owners_merged[HA_owners_merged['_merge'] != 'both']

# Filter matching and non-matching pairs to include only those within the 50 states
matching_pairs = {pair for pair in zip(matches['state'], matches['county']) if pair[0] in us_states}
non_matching_pairs = {pair for pair in zip(non_matches['state'].dropna(), non_matches['county'].dropna()) if pair[0] in us_states}

print("\nNon-matching County-State Pairs within 50 US States:")
print(non_matching_pairs)

# Get matching and non-matching pairs
matches = HA_renters_merged[HA_owners_merged['_merge'] == 'both']
non_matches = HA_renters_merged[HA_owners_merged['_merge'] != 'both']

# Filter matching and non-matching pairs to include only those within the 50 states
matching_pairs = {pair for pair in zip(matches['state'], matches['county']) if pair[0] in us_states}
non_matching_pairs = {pair for pair in zip(non_matches['state'].dropna(), non_matches['county'].dropna()) if pair[0] in us_states}

print("\nNon-matching County-State Pairs within 50 US States:")
print(non_matching_pairs)


Non-matching County-State Pairs within 50 US States:
{('AK', 'fairbanks north star'), ('VA', 'staunton'), ('CT', 'new haven msa 116054808880'), ('NH', 'rockingham msa 416047606450'), ('NC', 'eastern band of cherokee indians'), ('VA', 'alexandria'), ('VA', 'harrisonburg'), ('VA', 'radford'), ('PA', 'mckean'), ('GA', 'statewide'), ('IN', 'dekalb'), ('VA', 'virginia beach'), ('MD', 'wayside'), ('MT', 'blackfeet indian reservation'), ('VA', 'chesapeake'), ('VA', 'hampton'), ('VA', 'martinsville'), ('NM', 'mescalero tribe'), ('ND', 'standing rock sioux tribe of north south dakota'), ('AK', 'yukon koyukuk regional educational attendance area'), ('MD', 'prince georges'), ('IA', 'obrien'), ('GA', 'georgetown quitman'), ('AK', 'chignik lagoon'), ('VA', 'manassas'), ('CT', 'mohegan trust lands'), ('AK', 'yukon flats regional educational attendance area'), ('FL', 'brighton indian reservation'), ('VA', 'suffolk'), ('NH', 'hillsborough msa 456047605350'), ('VA', 'emporia'), ('AK', 'lower yukon reg

/tmp/ipykernel_2875/2528830118.py:57: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  matches = HA_renters_merged[HA_owners_merged['_merge'] == 'both']
/tmp/ipykernel_2875/2528830118.py:58: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  non_matches = HA_renters_merged[HA_owners_merged['_merge'] != 'both']


In [6]:
# Filter HA_owners to only include entries from the 50 U.S. states
HA_owners_filtered = HA_owners[HA_owners['state'].isin(us_states)]

# Filter HA_owners to only include entries from the 50 U.S. states
HA_renters_filtered = HA_renters[HA_renters['state'].isin(us_states)]

# Perform the merge to add 'countyCode' from fips_df to HA_owners
HA_owners_with_code = HA_owners_filtered.merge(
    fips_df[['countyCode', 'County Name', 'State Name']],
    left_on=['county', 'State Name'],
    right_on=['County Name', 'State Name'],
    how='left'
)

# Perform the merge to add 'countyCode' from fips_df to HA_owners
HA_renters_with_code = HA_renters_filtered.merge(
    fips_df[['countyCode', 'County Name', 'State Name']],
    left_on=['county', 'State Name'],
    right_on=['County Name', 'State Name'],
    how='left'
)

# Drop the extra 'County Name' column that came from the merge
HA_owners_with_code = HA_owners_with_code.drop(columns=['County Name'])

# Drop the extra 'County Name' column that came from the merge
HA_renters_with_code = HA_renters_with_code.drop(columns=['County Name'])

# Calculate the percentage of NaN values in the 'countyCode' column
na_percentage = HA_owners_with_code['countyCode'].isna().mean() * 100

# Print the percentage of NaN values
print(f"Percentage of NA values in 'countyCode' after merging: {na_percentage:.2f}%")

# Calculate the percentage of NaN values in the 'countyCode' column
na_percentage = HA_renters_with_code['countyCode'].isna().mean() * 100

# Print the percentage of NaN values
print(f"Percentage of NA values in 'countyCode' after merging: {na_percentage:.2f}%")

Percentage of NA values in 'countyCode' after merging: 4.76%
Percentage of NA values in 'countyCode' after merging: 5.06%


In [7]:
# Filtering out rows where 'countyCode' or 'disasterNumber' is NA
owners_filtered_df = HA_owners_with_code.dropna(subset=['countyCode', 'disasterNumber'])

# Aggregating the sum of 'approvedForFemaAssistance' by unique combinations of 'countyCode' and 'disasterNumber'
owners_result_df = owners_filtered_df.groupby(['countyCode', 'disasterNumber'], as_index=False)[['approvedForFemaAssistance', 'totalApprovedIhpAmount']].sum()

# Filtering out rows where 'countyCode' or 'disasterNumber' is NA
renters_filtered_df = HA_renters_with_code.dropna(subset=['countyCode', 'disasterNumber'])

# Aggregating the sum of 'approvedForFemaAssistance' by unique combinations of 'countyCode' and 'disasterNumber'
renters_result_df = renters_filtered_df.groupby(['countyCode', 'disasterNumber'], as_index=False)[['approvedForFemaAssistance', 'totalApprovedIhpAmount']].sum()

# Load Filtered Disasters

In [8]:
disasters_df = pd.read_csv('final_filtered_disasters.csv')
disasters_df['fullFIPS'] = disasters_df['fullFIPS'].astype(int).astype(str).str.zfill(5)

In [9]:
# Merging the result_df with disasters_df on 'countyCode' (from result_df) and 'fullFIPS' (from disasters_df),
# and 'disasterNumber'
owners_merged_df = disasters_df.merge(owners_result_df, left_on=['fullFIPS', 'disasterNumber'], 
                               right_on=['countyCode', 'disasterNumber'], how='left')

# Dropping the duplicate 'countyCode' column after the merge as it duplicates 'fullFIPS'
owners_merged_df = owners_merged_df.drop(columns=['countyCode'])

# Merging the result_df with disasters_df on 'countyCode' (from result_df) and 'fullFIPS' (from disasters_df),
# and 'disasterNumber'
renters_merged_df = disasters_df.merge(renters_result_df, left_on=['fullFIPS', 'disasterNumber'], 
                               right_on=['countyCode', 'disasterNumber'], how='left')

# Dropping the duplicate 'countyCode' column after the merge as it duplicates 'fullFIPS'
renters_merged_df = renters_merged_df.drop(columns=['countyCode'])

# Calculate the percentage of NaN values in the 'countyCode' column
na_percentage = owners_merged_df['fullFIPS'].isna().mean() * 100

# Print the percentage of NaN values
print(f"owners Percentage of NA values in 'countyCode' after merging: {na_percentage:.2f}%")

# Calculate the percentage of NaN values in the 'countyCode' column
na_percentage = renters_merged_df['fullFIPS'].isna().mean() * 100

# Print the percentage of NaN values
print(f"renters Percentage of NA values in 'countyCode' after merging: {na_percentage:.2f}%")

owners Percentage of NA values in 'countyCode' after merging: 0.00%
renters Percentage of NA values in 'countyCode' after merging: 0.00%


# Export Aid Data to CSV

In [10]:
owners_merged_df.to_csv("final_filtered_aid_owners.csv")
renters_merged_df.to_csv("final_filtered_aid_renters.csv")